Install Java, Hadoop & mrjob

In [1]:
!apt-get update -qq
!apt-get install -y openjdk-8-jdk-headless -qq
!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar -xf hadoop-3.3.6.tar.gz
import os, pathlib, sys, json, re, textwrap, math, itertools
os.environ["JAVA_HOME"]  = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
!pip install -q mrjob==0.7.4
!ln -sf /content/hadoop-3.3.6/bin/* /usr/bin/
!hadoop version

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u462-ga~us1-0ubuntu2~22.04.2_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u462-ga~us1-0ubuntu2~22.04.2) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u462-ga~us1-0ubuntu2~22.04.2_amd64.deb ...
Unpacking openjdk-8-jdk-headless:amd64 (8u462-ga~us1-0ubuntu2~22.04.2) ...
Setting up libxtst6:amd64 (2:1.2.3-1build4) ...
Setting up openjdk-8-jre-headless:amd64 (8u462-ga~

Get CSVs

In [2]:
!wget -q -O cruise.csv  https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/cruise.csv
!wget -q -O churn.csv   https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/customer_churn.csv
!wget -q -O ecommerce.csv https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/refs/heads/main/Dataset/e-com_customer.csv

In [3]:
from mrjob.job import MRJob
from mrjob.protocol import RawValueProtocol
from io import StringIO, BytesIO

Function for local runner

In [4]:
def run_local(mr_cls, sample_str):
    runner = mr_cls(args=['-r', 'inline', '--no-conf'])
    stdin  = BytesIO(sample_str.encode('utf-8'))
    with runner.make_runner() as r:
        r._stdin = stdin
        r.run()
        return [line.decode('utf-8').strip() for line in r.cat_output()]

Q1: For each Cruise line compute:
* ship_count
* avg_tonnage
* max_crew

Uses a combiner to lower shuffle volume.

In [5]:
%%file cruiseline_agg.py
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import RawValueProtocol
import csv, sys
class CruiseLineAgg(MRJob):
    """Q1: For each Cruise line compute
       (a) ship_count
       (b) avg_tonnage
       (c) max_crew
       Uses a combiner to lower shuffle volume."""
    OUTPUT_PROTOCOL = RawValueProtocol
    def mapper(self, _, line):
        if line.startswith('Ship_name'):
            return
        row = next(csv.reader([line]))
        line_name = row[1]
        tonnage   = float(row[3])
        crew      = float(row[8])
        yield line_name, (1, tonnage, crew)
    def combiner(self, line_name, vals):
        c, t_sum, c_max = 0, 0.0, 0.0
        for cnt, ton, cr in vals:
            c += cnt; t_sum += ton; c_max = max(c_max, cr)
        yield line_name, (c, t_sum, c_max)
    def reducer(self, line_name, vals):
        c, t_sum, c_max = 0, 0.0, 0.0
        for cnt, ton, cr in vals:
            c += cnt; t_sum += ton; c_max = max(c_max, cr)
        avg_ton = round(t_sum / c, 2)
        yield None, f"Cruise Line: {line_name} | Ship Count: {c} | Average Tonnage: {avg_ton:.2f} | Max Crew: {c_max}"
    def steps(self):
        return [MRStep(mapper=self.mapper,
                       combiner=self.combiner,
                       reducer=self.reducer)]
if __name__ == '__main__':
    CruiseLineAgg.run()

Writing cruiseline_agg.py


In [6]:
from cruiseline_agg import CruiseLineAgg

sample = (
    "Ship_name,Cruise_line,Age,Tonnage,passengers,length,cabins,passenger_density,crew\n"
    "Journey,Azamara,6,30.276999999999997,6.94,5.94,3.55,42.64,3.55\n"
    "Quest,Azamara,6,30.276999999999997,6.94,5.94,3.55,42.64,3.55\n"
    "Celebration,Carnival,26,47.262,14.86,7.22,7.43,31.8,6.7\n"
)

output_lines = run_local(CruiseLineAgg, sample)
for line in output_lines:
    print(line)


Cruise Line: Carnival | Ship Count: 1 | Average Tonnage: 47.26 | Max Crew: 6.7

Cruise Line: Azamara | Ship Count: 2 | Average Tonnage: 30.28 | Max Crew: 3.55


Q2: Multi-step job that filters to VIP firms (from distributed cache) then emits company-level churn rate to 4-decimal precision.

In [7]:
%%file churn_rate.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv, os
class CompanyChurnRate(MRJob):
    """Q2: Multi-step job that filters to VIP firms (from distributed
       cache) then emits company-level churn rate to 4-decimal precision."""
    FILES = ['vip_companies.txt']
    def configure_args(self):
        super().configure_args()
        self.add_file_arg('--vip', default='vip_companies.txt')
    def mapper_init(self):
        with open(self.options.vip) as f:
            self.vip = {l.strip() for l in f if l.strip()}
    def mapper(self, _, line):
        if line.lower().startswith('names'):
            return
        row = next(csv.reader([line]))
        company = row[-2]
        churned = int(row[-1]) == 1
        if company in self.vip:
            yield (company, 'TOTAL'), 1
            if churned:
              yield (company, 'CHURNED'), 1
    def reducer(self, key, vals):
        yield key[0], (key[1], sum(vals))
    def calc_rate(self, company, counts):
        counts_dict = dict(counts)
        total = counts_dict.get('TOTAL', 0)
        churned = counts_dict.get('CHURNED', 0)
        if total > 0:
            rate = churned / total
            yield None, f"Company: {company} | Churn Rate: {rate:.4f}"
        else:
            yield None, f"Company: {company} | Churn Rate: 0.0000"

    def steps(self):
        return [MRStep(mapper_init = self.mapper_init,
                       mapper=self.mapper,
                       reducer=self.reducer),
                MRStep(reducer=self.calc_rate)]
if __name__ == '__main__':
    CompanyChurnRate.run()

Writing churn_rate.py


In [8]:
from churn_rate import CompanyChurnRate

sample = (
    "Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn\n"
    "Cameron Williams,42.0,11066.8,0,7.22,8.0,2013-08-30 07:00:40,\"10265 Elizabeth Mission Barkerburgh, AK 89518\",Harvey LLC,1\n"
    "Kevin Mueller,41.0,11916.22,0,6.5,11.0,2013-08-13 00:38:46,\"6157 Frank Gardens Suite 019 Carloshaven, RI 17756\",Wilson PLC,1\n"
    "Eric Lozano,38.0,12884.75,0,6.67,12.0,2016-06-29 06:20:07,\"1331 Keith Court Alyssahaven, DE 90114\",\"Miller, Johnson and Wallace\",1\n"
    "Phillip White,42.0,8010.76,0,6.71,10.0,2014-04-22 12:43:12,\"13120 Daniel Mount Angelabury, WY 30645-4695\",Smith Inc,1\n"
)

output_lines = run_local(CompanyChurnRate, sample)
for line in output_lines:
    print(line)

null	"Company: Wilson PLC | Churn Rate: 1.0000"

null	"Company: Harvey LLC | Churn Rate: 1.0000"


Q3: Extract 2-letter state code from Address, sum Yearly Amount Spent, output top-5.

In [9]:
%%file state_spend.py
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import RawValueProtocol
import csv, sys
import re, heapq
class StateSpend(MRJob):
    """Q3: Extract 2-letter state code from Address,
       sum Yearly Amount Spent, output top-5."""
    OUTPUT_PROTOCOL = RawValueProtocol
    def mapper(self, _, line):
        if line.startswith('Email'):
            return
        row = next(csv.reader([line]))
        addr = row[1]
        amt_spend   = float(row[-1])
        m = re.compile(r',\s*([A-Z]{2})\s*\d{5}-\d{4}$').search(addr)
        if m:
          yield m.group(1), amt_spend
        else:
          yield "DEFAULT", amt_spend
    def reducer_sum(self, state, amt_spend):
        yield None, (sum(amt_spend), state)
    def reducer(self, _, state_amount):
        top = heapq.nlargest(5, state_amount)
        for amt, st in top:
            yield None, f"State: {st} | Amount Spent:{amt:.2f}"
    def steps(self):
        return [MRStep(mapper=self.mapper,
                       reducer=self.reducer_sum),
                MRStep(reducer=self.reducer)]
if __name__ == '__main__':
    StateSpend.run()

Writing state_spend.py


In [10]:
from state_spend import StateSpend

sample = (
  "Email,Address,Avatar,Avg Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent\n"
  "mstephenson@fernandez.com,\"835 Frank TunnelWrightmouth, MI 82180-9605\",Violet,34.49726772511229,12.65565114916675,39.57766801952616,4.0826206329529615,587.9510539684005\n"
  "hduke@hotmail.com,\"4547 Archer CommonDiazchester, CA 06566-8576\",DarkGreen,31.92627202636016,11.109460728682564,37.268958868297744,2.66403418213262,392.2049334443264\n"
  "pallen@yahoo.com,\"24645 Valerie Unions Suite 582Cobbborough, DC 99414-7564\",Bisque,33.000914755642675,11.330278057777512,37.110597442120856,4.104543202376424,487.54750486747207\n"
  "riverarebecca@gmail.com,\"1414 David ThroughwayPort Jason, OH 22070-1220\",SaddleBrown,34.30555662975554,13.717513665142507,36.72128267790313,3.120178782748092,581.8523440352177\n"
  "mstephens@davidson-herman.com,\"14023 Rodriguez PassagePort Jacobville, PR 37242-1057\",MediumAquaMarine,33.33067252364639,12.795188551078114,37.53665330059473,4.446308318351434,599.4060920457634\n"
  "alvareznancy@lucas.biz,\"645 Martha Park Apt. 611Jeffreychester, MN 67218-7250\",FloralWhite,33.871037879341976,12.026925339755056,34.47687762925054,5.493507201364199,637.102447915074\n"
)

output_lines = run_local(StateSpend, sample)
for line in output_lines:
    print(line)

State: MN | Amount Spent:637.10
State: PR | Amount Spent:599.41
State: MI | Amount Spent:587.95
State: OH | Amount Spent:581.85
State: DC | Amount Spent:487.55


Q4:
* filter ships with passenger_density>35.0
* compute per-line median length

In [11]:
%%file median_length.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv, statistics
class MedianLength(MRJob):
    """Q4: (1) filter ships with passenger_density>35.0
       (2) compute per-line median length"""
    def mapper(self, _, line):
        if line.startswith('Ship_name'):
            return
        row = next(csv.reader([line]))
        line_name = row[1]
        density   = float(row[7])
        if density > 35.0:
            yield line_name, float(row[5])
    def reducer(self, line_name, lengths):
        ls = sorted(lengths)
        med = statistics.median(ls)
        yield None, f"Cruise Line: {line_name} | Median Length: {med:.2f}"
    def steps(self):
        return [MRStep(mapper=self.mapper, reducer=self.reducer)]
if __name__ == '__main__':
    MedianLength.run()


Writing median_length.py


In [12]:
from median_length import MedianLength

sample = (
    "Ship_name,Cruise_line,Age,Tonnage,passengers,length,cabins,passenger_density,crew\n"
    "Journey,Azamara,6,30.276999999999997,6.94,5.94,3.55,42.64,3.55\n"
    "Quest,Azamara,6,30.276999999999997,6.94,5.94,3.55,42.64,3.55\n"
    "Celebration,Carnival,26,47.262,14.86,7.22,7.43,31.8,6.7\n"
)

output_lines = run_local(MedianLength, sample)
for line in output_lines:
    print(line)

null	"Cruise Line: Azamara | Median Length: 5.94"


Run every job on Hadoop inside Colab

In [13]:
%%bash
python cruiseline_aggs.py cruise.csv -r local --output-dir q1_out --no-conf --quiet
python churn_rate.py churn.csv -r local --vip vip_companies.txt --output-dir q2_out --no-conf --quiet
python state_spend.py ecommerce.csv -r local --output-dir q3_out --no-conf --quiet
python median_length.py cruise.csv -r local --output-dir q4_out --no-conf --quiet

echo "=== Q1: Cruise Line Aggregations ==="
cat q1_out/part-* > q1_full_output.txt
cat q1_full_output.txt

echo "\n=== Q2: Company Churn Rate (VIP) ==="
cat q2_out/part-* > q2_full_output.txt
cat q2_full_output.txt

echo "\n=== Q3: State-wise Yearly Spending (Top 5) ==="
cat q3_out/part-* > q3_full_output.txt
cat q3_full_output.txt

echo "\n=== Q4: Median Ship Length for Density > 35 ==="
cat q4_out/part-* > q4_full_output.txt
cat q4_full_output.txt

=== Q1: Cruise Line Aggregations ===
\n=== Q2: Company Churn Rate (VIP) ===
null	"Company: Harvey LLC | Churn Rate: 1.0000"
null	"Company: Martin and Sons | Churn Rate: 0.0000"
null	"Company: Wilson PLC | Churn Rate: 0.3333"
\n=== Q3: State-wise Yearly Spending (Top 5) ===
State: DEFAULT | Amount Spent:131829.76
State: NC | Amount Spent:3896.02
State: KS | Amount Spent:3874.21
State: OR | Amount Spent:3656.82
State: DE | Amount Spent:3369.33
\n=== Q4: Median Ship Length for Density > 35 ===
null	"Cruise Line: Azamara | Median Length: 5.94"
null	"Cruise Line: Carnival | Median Length: 9.52"
null	"Cruise Line: Celebrity | Median Length: 9.65"
null	"Cruise Line: Costa | Median Length: 8.28"
null	"Cruise Line: Crystal | Median Length: 7.86"
null	"Cruise Line: Cunard | Median Length: 9.64"
null	"Cruise Line: Disney | Median Length: 9.64"
null	"Cruise Line: Holland_American | Median Length: 7.77"
null	"Cruise Line: MSC | Median Length: 8.23"
null	"Cruise Line: Norwegian | Median Length: 9.00

python3: can't open file '/content/cruiseline_aggs.py': [Errno 2] No such file or directory
cat: 'q1_out/part-*': No such file or directory
